# Selenium Startup
Testes dinâmicos do Selenium pelo C#

## Instalação

In [ ]:
#r "nuget: Selenium.WebDriver, 4.10.0"
#r "nuget: Selenium.Support, 4.10.0"
#r "nuget: Selenium.WebDriver.ChromeDriver, 142.0.7444.16200"
#r "nuget: WebDriverManager, 2.17.6"

## Matar Processos
Para não ter que fazer isso manualmente um de cada vez ou em caso de processos zumbis

In [ ]:
using System.Diagnostics;

// Use com cuidado
void MatarProcessos(string nomeProcesso)
{
    var processos = Process.GetProcessesByName(nomeProcesso);
    foreach (var p in processos)
    {
        try 
        {
            p.Kill();
            Console.WriteLine($"Matou: {p.ProcessName} (ID: {p.Id})");
        }
        catch (Exception ex)
        {
            // Ignora erros de permissão ou se já morreu
        }
    }
}

// Fecha todos os chromes e drivers abertos
MatarProcessos("chromedriver");
MatarProcessos("chrome");

## Imports

In [33]:
// Sistema
using System.IO;
using System.Threading;
using System.Text.Json;

// Selenium
using OpenQA.Selenium;
using OpenQA.Selenium.Chrome;
using OpenQA.Selenium.Support.UI;

using WebDriverManager;
using WebDriverManager.DriverConfigs.Impl;

## Extensões
Simplificar algumas coisas

In [ ]:
static string GetText(this IWebElement element, string selector)
    => element.FindElement(By.CssSelector(selector)).Text;

static string GetSrcImage(this IWebElement element, string selector)
    => element.FindElement(By.CssSelector(selector)).GetAttribute("src");

static string ToJsonString(this object obj, JsonSerializerOptions? options = null)
    => JsonSerializer.Serialize(obj, options ?? new() { WriteIndented = true });

## Variáveis Globais

In [24]:
string tmpDir = Path.GetTempPath();
string userDataDir = Path.Combine(tmpDir, "tmp_chrome");

const int WAIT_TIMEOUT_SECONDS = 15;

const bool SHOW_CHROME = true;
const bool USE_FULL_SCREEN = true;

const bool USE_SERVER = false;
const bool USE_UNSAFE = false;

const bool DISABLE_EXTENSIONS = false;

## Inicialização do Chrome Driver

In [26]:
var options = new ChromeOptions();

// 1. Caminho do Perfil (O que você já tinha, CRUCIAL para manter login/certificados)
// Certifique-se que userDataDir existe ou o Chrome tem permissão de criar
options.AddArgument($"--user-data-dir={userDataDir}"); 

// 2. Estabilidade do Sistema & Compatibilidade (Docker/Server/Linux)
if (USE_SERVER)
{
    options.AddArgument("--no-sandbox"); // Essencial para rodar como root ou em alguns servers
    options.AddArgument("--disable-dev-shm-usage"); // Evita crash de memória compartilhada em containers
    options.AddArgument("--disable-gpu"); // Previne erros de renderização gráfica (comum em servidores)
    options.AddArgument("--remote-allow-origins=*"); // CORREÇÃO CRÍTICA para Chrome v111+ (evita erro de conexão websocket)
}

// 3. Rede e Certificados (Obrigatório para PJE/Sites Governo)
if (USE_UNSAFE)
{
    options.AddArgument("--ignore-certificate-errors"); // Ignora SSL vencido ou auto-assinado
    options.AddArgument("--ignore-ssl-errors=yes");
}

if (DISABLE_EXTENSIONS)
{
    options.AddArgument("--disable-extensions"); // A menos que você precise de uma extensão específica
}

// 4. Layout e Visualização
// Prefira window-size fixo ao invés de start-maximized para garantir consistência se rodar em Headless depois
if (USE_FULL_SCREEN)
{
    options.AddArgument("--start-maximized"); 
}
else
{
    options.AddArgument("--window-size=1920,1080"); 
}

// 5. Limpeza e Performance
options.AddArgument("--no-first-run");
options.AddArgument("--no-default-browser-check");
options.AddArgument("--disable-default-apps");
options.AddArgument("--disable-notifications"); // Bloqueia "Deseja receber notificações?"
options.AddArgument("--disable-popup-blocking"); // Permite popups (PJE usa muito para abrir PDFs)

// 6. (Opcional) "Disfarce" Básico
// Remove a barra amarela "Chrome está sendo controlado por software de teste"
options.AddExcludedArgument("enable-automation"); 
options.AddAdditionalOption("useAutomationExtension", false);

if (!SHOW_CHROME)
{
    options.AddArgument("--headless"); 
}

string driverPathCompleto = new DriverManager().SetUpDriver(new ChromeConfig());

// 2. O ChromeDriverService precisa apenas da PASTA, não do arquivo exe
string driverPasta = Path.GetDirectoryName(driverPathCompleto);

// 3. Criamos o serviço apontando explicitamente para onde o driver está
var service = ChromeDriverService.CreateDefaultService(driverPasta);

// Opcional: Esconder a janela preta do console do driver
service.HideCommandPromptWindow = true; 

// 4. Inicializa passando o SERVIÇO e as OPÇÕES
var driver = new ChromeDriver(service, options);

driver.Manage().Timeouts().ImplicitWait = TimeSpan.FromSeconds(WAIT_TIMEOUT_SECONDS);

var wait = new WebDriverWait(driver, TimeSpan.FromSeconds(15));

In [27]:
driver.Navigate().GoToUrl("https://www.saucedemo.com");

In [28]:
driver.FindElement(By.Id("user-name")).SendKeys("standard_user");
driver.FindElement(By.Id("password")).SendKeys("secret_sauce");
driver.FindElement(By.Id("login-button")).Click();

In [55]:
var elementList = driver.FindElements(By.ClassName("inventory_item"));

var items = elementList
    .Select<IWebElement, object>(
        element => new
        {
            title = element.GetText(".inventory_item_name"),
            description = element.GetText(".inventory_item_desc"),
            image = element.GetSrcImage("img.inventory_item_img"),
            price = double.Parse(
                element
                    .GetText(".inventory_item_price")
                    .Replace("$", "")
            ),
        }
    ).ToArray();

In [56]:
string json = items.ToJsonString();
int quantidade = items.Length;

Console.WriteLine($"Quantidade: {quantidade}");
Console.WriteLine($"Resultado: {json}");

Quantidade: 6
Resultado: [
  {
    "title": "Sauce Labs Backpack",
    "description": "carry.allTheThings() with the sleek, streamlined Sly Pack that melds uncompromising style with unequaled laptop and tablet protection.",
    "image": "https://www.saucedemo.com/static/media/sauce-backpack-1200x1500.0a0b85a385945026062b.jpg",
    "price": 29.99
  },
  {
    "title": "Sauce Labs Bike Light",
    "description": "A red light isn\u0027t the desired state in testing but it sure helps when riding your bike at night. Water-resistant with 3 lighting modes, 1 AAA battery included.",
    "image": "https://www.saucedemo.com/static/media/bike-light-1200x1500.37c843b09a7d77409d63.jpg",
    "price": 9.99
  },
  {
    "title": "Sauce Labs Bolt T-Shirt",
    "description": "Get your testing superhero on with the Sauce Labs bolt T-shirt. From American Apparel, 100% ringspun combed cotton, heather gray with red bolt.",
    "image": "https://www.saucedemo.com/static/media/bolt-shirt-1200x1500.c2599ac5f0